In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os,sys,inspect
import gc
from tqdm import tqdm
import random
import heapq

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from load import *
def eval_NDCG(true, pred):
    top_k = pred

    for i, item in enumerate(top_k, 1):
        if item == true:
            return 1 / np.log2(i+1)
    return 0

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Embedding, Flatten, Multiply
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def leakyrelu(x, factor=0.2):
    return tf.maximum(x, factor*x)

In [3]:
df = load_data('../data/ml-100k/u.data', threshold=3)
df = df[df['rating']==1].reset_index(drop=True)
tdf = pd.pivot_table(df, index='userId', values='rating', columns='movieId').fillna(0)

# 10개 이상 평가한 유저만 포함 => 0이 나오는 문제가 발생하여
cnt = tdf.sum(1)
df = df[df['userId'].isin(np.where(cnt >= 10)[0])].reset_index(drop=True)
tdf = pd.pivot_table(df, index='userId', values='rating', columns='movieId').fillna(0)
tdf.iloc[:,:] = 0

In [4]:
test_idx = []
for i in tdf.index:
    test_idx += list(np.random.choice(df[df['userId']==i].index, 1))
    
train = df.iloc[list(set(df.index)-set(test_idx)),:]
test = df.iloc[test_idx, :]

In [5]:
for uid, iid in zip(train['userId'].values, train['movieId'].values):
    tdf.loc[uid, iid] = 1
train =  tdf.copy()

In [6]:
class DAE(tf.keras.models.Model):
    def __init__(self, input_dim, latent_dim, lamda=1e-4):
        super().__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.lamda = lamda
        self.model = self.build()
        
    def compile(self, optimizer, loss_fn=None):
        super().compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn

        
    def build(self):
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        inputs = self.encoder.input
        outputs = self.decoder(self.encoder(inputs))
        
        return Model(inputs, outputs)
    
    def build_encoder(self):
        inputs = Input(shape = (self.input_dim, ))
        
        encoder = Sequential()
        encoder.add(Dropout(0.2))
        encoder.add(Dense(self.latent_dim, activation='tanh'))
        
        outputs = encoder(inputs)
        
        return Model(inputs, outputs)
    
    def build_decoder(self):
        inputs = Input(shape = (self.latent_dim, ))
        
        encoder = Sequential()
        encoder.add(Dense(self.input_dim, activation='sigmoid'))
        
        outputs = encoder(inputs)
        
        return Model(inputs, outputs)
    
    def train_step(self, x):
        with tf.GradientTape() as tape:
            pred = self.model(x)
            
            rec_loss = tf.losses.binary_crossentropy(x, pred)
            loss = rec_loss

        grads = tape.gradient(loss, self.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
        
        return {'loss': loss}
    

In [7]:
loader = tf.data.Dataset.from_tensor_slices(train.values)
loader = loader.batch(32, drop_remainder=True).shuffle(len(df))

In [8]:
model = DAE(train.shape[1], 200)
model.compile(optimizer=tf.optimizers.Adam())

In [9]:
model.fit(loader,
           epochs = 25)

Epoch 1/25
28/28 [==============================] - 0s 2ms/step - loss: 0.4157
Epoch 2/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1803
Epoch 3/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1597
Epoch 4/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1503
Epoch 5/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1424
Epoch 6/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1384
Epoch 7/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1312
Epoch 8/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1280
Epoch 9/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1241
Epoch 10/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1209
Epoch 11/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1177
Epoch 12/25
28/28 [==============================] - 0s 2ms/step - loss: 0.1158
Epoch 13/25
28/28 [==============================

In [10]:
top_k = 10

scores = []
for idx, i in tqdm(enumerate(train.index)):
    item_to_pred = {item: pred for item, pred in zip(train.columns, model.model.predict(train.values)[idx])}
    test_ = test[(test['userId']==i) & (test['rating']==1)]['movieId'].values
    items = list(np.random.choice(list(filter(lambda x: x not in np.argwhere(train.values[idx]).flatten(), item_to_pred.keys())), 100)) + list(test_)
    top_k_items = heapq.nlargest(top_k, items, key=item_to_pred.get)
    
    score = eval_NDCG(test_, top_k_items)
    scores.append(score)
    
np.mean(scores)

896it [02:32,  5.87it/s]


0.43863281360932843